In [1]:
import os
import sys
#
import tensorflow as tf
#tf.keras.backend.set_floatx('float16')
#
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.layers import (
    Activation, AveragePooling2D, BatchNormalization, Conv2D, Conv2DTranspose,
    Dense, Dropout, Flatten, Input, LeakyReLU, ReLU, UpSampling2D)
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from time import time

import numpy as np
import cv2
import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from tensorflow.keras import Input
#from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Conv2D
#
from matplotlib import pyplot
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img,save_img
#from keras.utils.vis_utils import plot_model
from copy import deepcopy

from skimage.color import rgb2lab, lab2rgb
from PIL import Image


2023-09-26 09:33:58.053292: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 09:33:59.245208: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# conversion lab & rgb
def lab_to_rgb(img_lab):
    img_lab = img_lab.numpy()
    L = (img_lab[:,:,:,0] + 1.) * 50. 
    L = L[..., np.newaxis]
    ab = img_lab[:,:,:,1:]  * 110
    img_lab = np.concatenate((L,ab),axis=3)
    #
    img = lab2rgb(img_lab).astype("float32")
    img = (img * 255).astype(np.uint8)
    return img

def rgb_to_lab(img):
    img_lab = rgb2lab(img).astype("float32") # Converting RGB to L*a*b    # float16 to optimize ?
    # Between -1 and 1
    L  = img_lab[:,:,0]  / 50. - 1. 
    ab = img_lab[:,:,1:]  / 110. # Between -1 and 1 
    return L, ab

In [3]:
# Preload dataset L&ab
# &
# create npz file for GAN input
from os import listdir
from numpy import asarray
from numpy import vstack
from numpy import savez_compressed
import os
from random import shuffle, seed

BATCH_SIZE = 4
#SHUFFLE_BUFFER_SIZE = 100

path_mem = './Dataset_color_red/V_Urban_Launcher_dts/img_Urban_new'
path_old = './Dataset_color_red/V_Urban_Launcher_dts/img_Urban_old'
save_path = './Dataset_color_red/V_Urban_Launcher/'
#
dataset_size = len([i for i in os.listdir(path_mem)])
limit = (dataset_size // BATCH_SIZE)*BATCH_SIZE
print(dataset_size,limit)
LIST=[]

i=0

IMAGE=os.listdir(path_mem)
IMAGE.sort()
IMAGE=np.array(IMAGE)

OLD=os.listdir(path_old)
OLD.sort()
OLD=np.array(OLD)

640 640


In [5]:
# Shuffle dataset
seed(11223355)
shuffle(IMAGE)

seed(11223355)
shuffle(OLD)

In [6]:
def load_images_path(path):
    im_list= list()
    # enumerate filenames in directory, assume all are images
    for filename in os.listdir(path):
        im_list.append(os.path.join(path,filename))
        print(filename)
    return asarray(im_list)

def load_images(path):
    pixels = Image.open(path).convert("RGB")
    # Convert RGB to L & ab
    L_img, ab_img = rgb_to_lab(pixels)
    L_img = L_img[..., np.newaxis]
    return [np.asarray(L_img), np.asarray(ab_img)]

def create_dataset(LIST,BATCH_SIZE):
    List_dataset = tf.data.Dataset.from_tensor_slices(LIST)
    tf.data.Dataset.save(List_dataset, save_path +'Dataset_'+str(i))
        
    for L in List_dataset:
        print('dataset_'+str(i)+' saved', tf.shape(L))
        break

In [7]:
# Generate dataset file every 100 images 
# because of memory constraints, then 
# restart kernel and merge + save the final file

for filename,filename_old in zip(IMAGE,OLD):
    print(filename,filename_old)
    if ((i % 100 == 0) and (i !=0)):
        create_dataset(LIST,BATCH_SIZE)
        LIST=[]

    # LAB image new
    path = os.path.join(path_mem,filename)
    L_data, ab_data = load_images(path)
    # old
    path_o = os.path.join(path_old,filename_old)
    L_bin, ab_old_data = load_images(path_o)
    #
    LIST.append(np.concatenate((L_data, ab_data, ab_old_data), axis=2))
    
    i+=1

    if (i==limit):
        create_dataset(LIST,BATCH_SIZE)
        LIST=[]


img_cropped4_20.png img_cropped4_20.png
img_cropped1_16.png img_cropped1_16.png
img_cropped3_32.png img_cropped3_32.png
img_cropped2_0.png img_cropped2_0.png
im_283.png im_283.png
im_364.png im_364.png
im_293.png im_293.png
img_cropped3_28.png img_cropped3_28.png
im_393.png im_393.png
im_018.png im_018.png
im_329.png im_329.png
im_226.png im_226.png
im_304.png im_304.png
im_050.jpg im_050.jpg
im_493.jpg im_493.jpg
im_263.png im_263.png
im_133.jpg im_133.jpg
img_cropped3_2.png img_cropped3_2.png
im_188.jpg im_188.jpg
gearth_13_09_007_org.jpg gearth_13_09_007_org.jpg
im_451.png im_451.png
im_160.jpg im_160.jpg
im_026.jpg im_026.jpg
img_cropped1_26.png img_cropped1_26.png
im_181.jpg im_181.jpg
im_040.JPG im_040.JPG
im_401.png im_401.png
im_082.jpg im_082.jpg
im_218.png im_218.png
img_cropped3_8.png img_cropped3_8.png
dd_org.jpg dd_org.jpg
img_cropped1_2.png img_cropped1_2.png
im_087.jpg im_087.jpg
im_244.png im_244.png
im_395.png im_395.png
im_052.JPG im_052.JPG
im_474.png im_474.png
img_

2023-09-26 09:26:17.722743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 09:26:17.918219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 09:26:17.918439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

dataset_100 saved tf.Tensor([512 512   5], shape=(3,), dtype=int32)
img_maps_18_09_005.png img_maps_18_09_005.png
im_481.jpg im_481.jpg
img_cropped1_14.png img_cropped1_14.png
im_010.jpg im_010.jpg
im_116.jpg im_116.jpg
im_310.png im_310.png
gearth_13_09_005_org.jpg gearth_13_09_005_org.jpg
gearth_13_09_014_org.jpg gearth_13_09_014_org.jpg
im_173.jpg im_173.jpg
im_341.png im_341.png
im_461.png im_461.png
im_106.jpg im_106.jpg
img_18_09_512_002.jpg img_18_09_512_002.jpg
im_261.png im_261.png
im_183.jpg im_183.jpg
img_cropped2_14.png img_cropped2_14.png
im_335.png im_335.png
img_cropped3_34.png img_cropped3_34.png
im_471.jpg im_471.jpg
img_croppedR_7.png img_croppedR_7.png
im_439.png im_439.png
im_458.png im_458.png
im_298.png im_298.png
im_333.png im_333.png
img_cropped4_28.png img_cropped4_28.png
im_391.png im_391.png
im_340.png im_340.png
im_221.png im_221.png
im_339.png im_339.png
im_402.png im_402.png
im_121.jpg im_121.jpg
gearth_13_09_001_org.jpg gearth_13_09_001_org.jpg
img_croppe

2023-09-26 09:27:11.360773: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 524288000 exceeds 10% of free system memory.


dataset_200 saved tf.Tensor([512 512   5], shape=(3,), dtype=int32)
img_cropped2_28.png img_cropped2_28.png
im_094.jpg im_094.jpg
im_039.JPG im_039.JPG
im_436.png im_436.png
img_maps_18_09_010.png img_maps_18_09_010.png
im_315.png im_315.png
im_251.png im_251.png
im_008.jpg im_008.jpg
im_331.png im_331.png
im_423.png im_423.png
img_cropped3_20.png img_cropped3_20.png
im_103.jpg im_103.jpg
im_286.png im_286.png
image_00000019_2.png_TGAN_3_aj_22_09.jpg image_00000019_2_aj_22_09.png
im_476.jpg im_476.jpg
im_075.JPG im_075.JPG
im_171.jpg im_171.jpg
im_231.png im_231.png
im_241.png im_241.png
im_019.png im_019.png
im_270.png im_270.png
im_222.png im_222.png
im_162.jpg im_162.jpg
im_495.jpg im_495.jpg
im_025.jpg im_025.jpg
im_089.png im_089.png
im_001.jpg im_001.jpg
img_18_09_512_004.jpeg img_18_09_512_004.jpeg
im_155.jpg im_155.jpg
im_367.png im_367.png
img_gearth_18_09_013.jpg img_gearth_18_09_013.jpg
im_384.png im_384.png
img_croppedR_5.png img_croppedR_5.png
im_410.png im_410.png
im_216.

dataset_500 saved tf.Tensor([512 512   5], shape=(3,), dtype=int32)
img_croppedR_27.png img_croppedR_27.png
im_490.jpg im_490.jpg
im_020.png im_020.png
img_cropped3_4.png img_cropped3_4.png
im_114.jpg im_114.jpg
im_291.png im_291.png
img_cropped1_4.png img_cropped1_4.png
im_084.png im_084.png
img_cropped4_30.png img_cropped4_30.png
im_414.png im_414.png
im_186.jpg im_186.jpg
im_110.jpg im_110.jpg
im_206.jpg im_206.jpg
im_292.png im_292.png
img_cropped3_12.png img_cropped3_12.png
im_088.png im_088.png
im_064.JPG im_064.JPG
im_272.png im_272.png
im_120.jpg im_120.jpg
im_492.png im_492.png
im_228.png im_228.png
img_cropped4_34.png img_cropped4_34.png
im_491.jpg im_491.jpg
img_cropped4_4.png img_cropped4_4.png
im_062.JPG im_062.JPG
im_105.jpg im_105.jpg
im_029.jpg im_029.jpg
im_426.png im_426.png
im_153.jpg im_153.jpg
im_163.jpg im_163.jpg
gearth_13_09_021_org.jpg gearth_13_09_021_org.jpg
im_257.png im_257.png
im_158.jpg im_158.jpg
im_065.JPG im_065.JPG
img_cropped4_32.png img_cropped4_32.

**MERGE**

In [4]:
path_List = './Dataset_color_red/V_Urban_Launcher/'

List_path=os.listdir(path_List)
List_path.sort()
List_path=np.array(List_path)

d=1

for L in List_path:
    print(L)
    L_data = tf.data.Dataset.load(os.path.join(path_List,L))
    if d: 
        L_dataset = L_data

    else:
        L_dataset = L_dataset.concatenate(L_data)
    d=0
    
print('---------- FIN ----------')
for j,L in enumerate(L_dataset):
    pass
print('Taille finale :',j+1)

Dataset_100


2023-09-26 09:34:06.322726: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 09:34:06.599210: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 09:34:06.599425: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Dataset_200
Dataset_300
Dataset_400
Dataset_500
Dataset_600
Dataset_640
---------- FIN ----------
Taille finale : 640


**SAVE**

In [5]:
tf.data.Dataset.save(L_dataset, './Dataset_color_red/V_Urban_Launcher/Dataset_VU')

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 13255797570795942544
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 1282834310575036635
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing t